## Prepare data from Microsoft Open Dataset

https://azure.microsoft.com/en-us/services/open-datasets/catalog/sample-oj-sales-simulated

In [3]:
data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")

In [4]:
#Write to local delta for fast reading
data.write.format("delta").saveAsTable("OJ_Sales_Data")

In [5]:
%sql select * from OJ_Sales_Data

WeekStarting,Store,Brand,Quantity,Advert,Price,Revenue
1990-06-14,1094,minute.maid,17892,1,2.09,37394.28
1990-06-21,1094,minute.maid,14053,1,2.45,34429.850000000006
1990-06-28,1094,minute.maid,17341,1,2.47,42832.270000000004
1990-07-05,1094,minute.maid,17194,1,2.42,41609.479999999996
1990-07-12,1094,minute.maid,17945,1,2.39,42888.55
1990-07-19,1094,minute.maid,17371,1,2.3,39953.299999999996
1990-07-26,1094,minute.maid,9825,1,2.36,23187.0
1990-08-02,1094,minute.maid,10849,1,2.58,27990.420000000002
1990-08-09,1094,minute.maid,12084,1,2.0,24168.0
1990-08-16,1094,minute.maid,10484,1,2.32,24322.879999999997


In [6]:
%sql select count (distinct store, brand) from OJ_Sales_Data 

"count(DISTINCT store, brand)"
300


In [7]:
%sql select distinct brand from OJ_Sales_Data

brand
minute.maid
dominicks
tropicana


## Pre-training exersize

1. Read about Pandas Function APIs: https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/pandas-function-apis
2. Answer following questions:
- What is the advantage of this technology vs. regular Python UDF?
- What is the role of Apache Arrow in this?
- What is the use of iterator and yield vs. regular list and return?

Using the OJ sales dataset above, use Pandas Function APIs, pick out for each store and brand the best selling week in the form of week_number-yyyy.
The result set look like this:

In [11]:
import pandas as pd
result_sample= pd.DataFrame({"store": [1066, 1067, 1068],'Brand':['dominicks', 'tropicana','tropicana'],"Best_Selling_Week": ['23-1992', '24-1991','24-1991']})
display(result_sample)

store,Brand,Best_Selling_Week
1066,dominicks,23-1992
1067,tropicana,24-1991
1068,tropicana,24-1991


#Optional reading: we'll  following forecast models from the Many Models repo

In [13]:
https://github.com/microsoft/solution-accelerator-many-models/blob/master/Custom_Script/scripts/timeseries_utilities.py
https://github.com/microsoft/solution-accelerator-many-models/blob/master/Custom_Script/scripts/train.py
https://github.com/microsoft/solution-accelerator-many-models/blob/master/Custom_Script/scripts/forecast.py